# What the Text Editor Tool Can Do

### The text editor tool provides Claude with a comprehensive set of file manipulation capabilities:

- View file or directory contents
- View specific ranges of lines in a file
- Replace text in a file
- Create new files
- Insert text at specific lines in a file
- Undo recent edits to files

### Why Use the Text Editor Tool?

You might wonder why this tool exists when modern code editors already have AI assistants built in. The text editor tool becomes valuable in scenarios where:

- You're building applications that need to programmatically edit files
- You're working in environments without access to full-featured code editors
- You want to integrate file editing capabilities directly into your Claude-powered applications

Essentially, the text editor tool lets you replicate much of the functionality of a fancy AI-powered code editor within your own applications, giving you fine-grained control over how Claude interacts with your file system.

# dependencies

In [11]:
from anthropic import Anthropic

from dotenv import load_dotenv
import os   
load_dotenv()

import logging
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
    )
logger = logging.getLogger(__name__)

try:
    client = Anthropic(
        api_key=os.getenv("ANTHROPIC_API_KEY")
    )
    logger.info("Anthropic client initialized successfully.")
except Exception as e:
    logger.error(f"Failed to initialize Anthropic client: {e}")
    raise e

2025-08-08 12:33:34 - __main__ - INFO - Anthropic client initialized successfully.


# global config

In [12]:
MODEL="claude-3-5-sonnet-20241022"
TEMPERATURE=1

# helper functions

In [13]:
def add_user_message(messages, message_content):
    if isinstance(message_content, list):
        user_message = {
            "role": "user",
            "content": message_content
        }
    else:
        user_message = {
            "role": "user",
            "content": [{"type": "text", "text": str(message_content)}]}

    messages.append(user_message)


def add_assistant_message(messages, message_content):
    if isinstance(message_content, list):
        assistant_message = {
            "role": "assistant",
            "content": message_content
        }
    elif hasattr(message_content, "content"):
        content_list = []
        for block in message_content.content:
            if block.type == "text":
                content_list.append({"type": "text", "text": block.text})
            elif block.type == "tool_use":
                content_list.append({
                    "type": "tool_use",
                    "id": block.id,
                    "name": block.name,
                    "input": block.input
                })
        assistant_message = {
            "role": "assistant",
            "content": content_list
        }
    else:
        assistant_message = {
            "role": "assistant",
            "content": [{"type": "text", "text": message_content}]
        }

    messages.append(assistant_message)

def chat(messages, model=MODEL, temperature=TEMPERATURE, system=None, stop_sequences=None, tools=None, tool_choice=None, betas=[]):
    try:
        params = {
            "model": model,
            "messages": messages,
            "temperature": temperature,
            "max_tokens": 1000,
            "stop_sequences": stop_sequences,
        }
        if system:
            params["system"] = system

        if tools:
            params["tools"] = tools

        if tool_choice:
            params["tool_choice"] = tool_choice

        if betas:
            params["betas"] = betas

        if stop_sequences:
            params["stop_sequences"] = stop_sequences

        # Use client.beta.messages.stream for streaming
        return client.messages.create(**params)
        
    except Exception as e:
        logger.error(f"Chat streaming failed: {e}")
        raise e
    
def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )

# Implementation of the TextEditorTool

In [14]:
import os
import shutil
from typing import Optional, List


class TextEditorTool:
    def __init__(self, base_dir: str = "", backup_dir: str = ""):
        self.base_dir = base_dir or os.getcwd()
        self.backup_dir = backup_dir or os.path.join(self.base_dir, ".backups")
        os.makedirs(self.backup_dir, exist_ok=True)

    def _validate_path(self, file_path: str) -> str:
        abs_path = os.path.normpath(os.path.join(self.base_dir, file_path))
        if not abs_path.startswith(self.base_dir):
            raise ValueError(
                f"Access denied: Path '{file_path}' is outside the allowed directory"
            )
        return abs_path

    def _backup_file(self, file_path: str) -> str:
        if not os.path.exists(file_path):
            return ""
        file_name = os.path.basename(file_path)
        backup_path = os.path.join(
            self.backup_dir, f"{file_name}.{os.path.getmtime(file_path):.0f}"
        )
        shutil.copy2(file_path, backup_path)
        return backup_path

    def _restore_backup(self, file_path: str) -> str:
        file_name = os.path.basename(file_path)
        backups = [
            f
            for f in os.listdir(self.backup_dir)
            if f.startswith(file_name + ".")
        ]
        if not backups:
            raise FileNotFoundError(f"No backups found for {file_path}")

        latest_backup = sorted(backups, reverse=True)[0]
        backup_path = os.path.join(self.backup_dir, latest_backup)

        shutil.copy2(backup_path, file_path)
        return f"Successfully restored {file_path} from backup"

    def _count_matches(self, content: str, old_str: str) -> int:
        return content.count(old_str)

    def view(
        self, file_path: str, view_range: Optional[List[int]] = None
    ) -> str:
        try:
            abs_path = self._validate_path(file_path)

            if os.path.isdir(abs_path):
                try:
                    return "\n".join(os.listdir(abs_path))
                except PermissionError:
                    raise PermissionError(
                        "Permission denied. Cannot list directory contents."
                    )

            if not os.path.exists(abs_path):
                raise FileNotFoundError("File not found")

            with open(abs_path, "r", encoding="utf-8") as f:
                content = f.read()

            if view_range:
                start, end = view_range
                lines = content.split("\n")

                if end == -1:
                    end = len(lines)

                selected_lines = lines[start - 1 : end]

                result = []
                for i, line in enumerate(selected_lines, start):
                    result.append(f"{i}: {line}")

                return "\n".join(result)
            else:
                lines = content.split("\n")
                result = []
                for i, line in enumerate(lines, 1):
                    result.append(f"{i}: {line}")

                return "\n".join(result)

        except UnicodeDecodeError:
            raise UnicodeDecodeError(
                "utf-8",
                b"",
                0,
                1,
                "File contains non-text content and cannot be displayed.",
            )
        except ValueError as e:
            raise ValueError(str(e))
        except PermissionError:
            raise PermissionError("Permission denied. Cannot access file.")
        except Exception as e:
            raise type(e)(str(e))

    def str_replace(self, file_path: str, old_str: str, new_str: str) -> str:
        try:
            abs_path = self._validate_path(file_path)

            if not os.path.exists(abs_path):
                raise FileNotFoundError("File not found")

            with open(abs_path, "r", encoding="utf-8") as f:
                content = f.read()

            match_count = self._count_matches(content, old_str)

            if match_count == 0:
                raise ValueError(
                    "No match found for replacement. Please check your text and try again."
                )
            elif match_count > 1:
                raise ValueError(
                    f"Found {match_count} matches for replacement text. Please provide more context to make a unique match."
                )

            # Create backup before modifying
            self._backup_file(abs_path)

            # Perform the replacement
            new_content = content.replace(old_str, new_str)

            with open(abs_path, "w", encoding="utf-8") as f:
                f.write(new_content)

            return "Successfully replaced text at exactly one location."

        except ValueError as e:
            raise ValueError(str(e))
        except PermissionError:
            raise PermissionError("Permission denied. Cannot modify file.")
        except Exception as e:
            raise type(e)(str(e))

    def create(self, file_path: str, file_text: str) -> str:
        try:
            abs_path = self._validate_path(file_path)

            # Check if file already exists
            if os.path.exists(abs_path):
                raise FileExistsError(
                    "File already exists. Use str_replace to modify it."
                )

            # Create parent directories if they don't exist
            os.makedirs(os.path.dirname(abs_path), exist_ok=True)

            # Create the file
            with open(abs_path, "w", encoding="utf-8") as f:
                f.write(file_text)

            return f"Successfully created {file_path}"

        except ValueError as e:
            raise ValueError(str(e))
        except PermissionError:
            raise PermissionError("Permission denied. Cannot create file.")
        except Exception as e:
            raise type(e)(str(e))

    def insert(self, file_path: str, insert_line: int, new_str: str) -> str:
        try:
            abs_path = self._validate_path(file_path)

            if not os.path.exists(abs_path):
                raise FileNotFoundError("File not found")

            # Create backup before modifying
            self._backup_file(abs_path)

            with open(abs_path, "r", encoding="utf-8") as f:
                lines = f.readlines()

            # Handle line endings
            if lines and not lines[-1].endswith("\n"):
                new_str = "\n" + new_str

            # Insert at the beginning if insert_line is 0
            if insert_line == 0:
                lines.insert(0, new_str + "\n")
            # Insert after the specified line
            elif insert_line > 0 and insert_line <= len(lines):
                lines.insert(insert_line, new_str + "\n")
            else:
                raise IndexError(
                    f"Line number {insert_line} is out of range. File has {len(lines)} lines."
                )

            with open(abs_path, "w", encoding="utf-8") as f:
                f.writelines(lines)

            return f"Successfully inserted text after line {insert_line}"

        except ValueError as e:
            raise ValueError(str(e))
        except PermissionError:
            raise PermissionError("Permission denied. Cannot modify file.")
        except Exception as e:
            raise type(e)(str(e))

    def undo_edit(self, file_path: str) -> str:
        try:
            abs_path = self._validate_path(file_path)

            if not os.path.exists(abs_path):
                raise FileNotFoundError("File not found")

            return self._restore_backup(abs_path)

        except ValueError as e:
            raise ValueError(str(e))
        except FileNotFoundError:
            raise FileNotFoundError("No previous edits to undo")
        except PermissionError:
            raise PermissionError("Permission denied. Cannot restore file.")
        except Exception as e:
            raise type(e)(str(e))

# Process Tool Call Requests

In [15]:
import json

text_editor_tool = TextEditorTool()


def run_tool(tool_name, tool_input):
    if tool_name in ["str_replace_editor", "str_replace_based_edit_tool"]:  
        command = tool_input["command"]
        if command == "view":
            logger.info(f"Viewing file: {tool_input['path']}")
            return text_editor_tool.view(
                tool_input["path"], tool_input.get("view_range")
            )
        elif command == "str_replace":
            logger.info(f"Replacing text in file: {tool_input['path']}")
            return text_editor_tool.str_replace(
                tool_input["path"], tool_input["old_str"], tool_input["new_str"]
            )
        elif command == "create":
            logger.info(f"Creating file: {tool_input['path']}")
            return text_editor_tool.create(
                tool_input["path"], tool_input["file_text"]
            )
        elif command == "insert":
            logger.info(f"Inserting text in file: {tool_input['path']}")
            return text_editor_tool.insert(
                tool_input["path"],
                tool_input["insert_line"],
                tool_input["new_str"],
            )
        elif command == "undo_edit":
            logger.info(f"Undoing edit in file: {tool_input['path']}")
            return text_editor_tool.undo_edit(tool_input["path"])
        else:
            raise Exception(f"Unknown text editor command: {command}")
    else:
        raise Exception(f"Unknown tool name: {tool_name}")


def run_tools(message):
    tool_requests = [
        block for block in message.content if block.type == "tool_use"
    ]
    tool_result_blocks = []

    for tool_request in tool_requests:
        try:
            tool_output = run_tool(tool_request.name, tool_request.input)
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": json.dumps(tool_output),
                "is_error": False,
            }
            logger.info(f"Tool {tool_request.name} executed successfully with output: {tool_output}")
        except Exception as e:
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": f"Error: {e}",
                "is_error": True,
            }

        tool_result_blocks.append(tool_result_block)

    return tool_result_blocks

# Make the text edit schema based on the model version being used

In [16]:
def get_text_edit_schema(model):
    if model.startswith("claude-opus-4"):
        return {
            "type": "text_editor_20250728",
            "name": "str_replace_editor",
        }
    elif model.startswith("claude-3-7-sonnet"):
        return {
            "type": "text_editor_20250124",
            "name": "str_replace_editor",
        }
    elif model.startswith("claude-3-5-sonnet-20241022"):  
        return {
            "type": "text_editor_20250124",  
            "name": "str_replace_editor",    
        }
    elif model.startswith("claude-3-5-sonnet"):
        return {
            "type": "text_editor_20250429",
            "name": "str_replace_based_edit_tool",
        }
    else:
        raise ValueError(
            f"Editor schema version not known for model: {model}. Reference Anthropic docs for the correct schema version."
        )

# Run the conversation in a loop until the model doesn't ask for a tool use

In [17]:
def run_conversation(messages):
    while True:
        response = chat(
            messages,
            tools=[get_text_edit_schema(MODEL)],
        )

        add_assistant_message(messages, response)
        print(text_from_message(response))

        if response.stop_reason != "tool_use":
            break

        tool_results = run_tools(response)
        add_user_message(messages, tool_results)

    return messages

# test case

In [18]:
PROMPT = """
I need help creating a simple Python script for a calculator. Please:
    
    1. First, check what files are in the current directory
    2. Create a new file called "calculator.py" 
    3. Add a basic calculator function that can add two numbers
    4. Then view the file to confirm it was created correctly
    5. Add a multiplication function by inserting it after the add function
    6. Finally, show me the complete file
    
    Make the calculator functions simple but functional.
"""

### The expected flow will be:

1. Claude uses view to see current directory
2. Claude uses create to make "calculator.py" with an add function
3. Claude uses view to confirm the file was created
4. Claude uses insert to add a multiply function after the add function
5. Claude uses view to show the complete file

# main function

In [19]:
def main():
    messages = []
    
    add_user_message(
        messages,
        PROMPT,
    )

    run_conversation(messages)

In [20]:
main()

2025-08-08 12:33:38 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-08-08 12:33:38 - __main__ - INFO - Viewing file: /repo


I'll help you create the calculator script step by step.

1. First, let's check what files are in the current directory:


2025-08-08 12:33:41 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-08-08 12:33:41 - __main__ - INFO - Viewing file: .
2025-08-08 12:33:41 - __main__ - INFO - Tool str_replace_editor executed successfully with output: text_editor_tool.ipynb
using_multiple_tools.ipynb
fine_graded_tool_calling.ipynb
web_search_tool.ipynb
.backups
NOTES.md
assets


I notice we need to check the correct directory. Let me modify the path:


2025-08-08 12:33:45 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-08-08 12:33:45 - __main__ - INFO - Creating file: calculator.py
2025-08-08 12:33:45 - __main__ - INFO - Tool str_replace_editor executed successfully with output: Successfully created calculator.py


...

5. Now, let's add the multiplication function after the add function. We'll insert it after line 3:


2025-08-08 12:33:57 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-08-08 12:33:57 - __main__ - INFO - Viewing file: calculator.py
2025-08-08 12:33:57 - __main__ - INFO - Tool str_replace_editor executed successfully with output: 1: def add(a, b):
2:     """Add two numbers and return the result."""
3:     return a + b
4: 
5: def multiply(a, b):
6:     """Multiply two numbers and return the result."""
7:     return a * b
8: 


6. Finally, let's view the complete file:


2025-08-08 12:34:00 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


The calculator.py file has been created successfully with two functions:
1. `add(a, b)` - takes two numbers and returns their sum
2. `multiply(a, b)` - takes two numbers and returns their product

The functions are simple but functional, with proper docstrings explaining what they do. You can use these functions by importing the calculator.py file and calling the functions with two numbers as arguments.
